# Preprocessing for Machine Learning in Python
#### Course Description
This course covers the basics of how and when to perform data preprocessing. This essential step in any machine learning project is when you get your data ready for modeling. Between importing and cleaning your data and fitting your machine learning model is when preprocessing comes into play. You'll learn how to standardize your data so that it's in the right form for your model, create new features to best leverage the information in your dataset, and select the best features to improve your model fit. Finally, you'll have some practice preprocessing by getting a dataset on UFO sightings ready for modeling.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB



# 1. Introduction to Data Preprocessing

In this chapter you'll learn exactly what it means to preprocess data. You'll take the first steps in any preprocessing journey, including exploring data types and dealing with missing data.

### Missing data - rows
Taking a look at the volunteer dataset again, we want to drop rows where the category_desc column values are missing. We're going to do this using boolean indexing, by checking to see if we have any null values, and then filtering the dataset so that we only have rows with those values.

#### Instructions

- Check how many values are missing in the category_desc column using isnull() and sum().
- Subset the volunteer dataset by indexing by where category_desc is notnull(), and store in a new variable called volunteer_subset.
- Take a look at the .shape attribute of the new dataset, to verify it worked correctly.


In [2]:
vol_cols = ['opportunity_id', 'content_id', 'vol_requests', 'event_time', 'title',
       'hits', 'summary', 'is_priority', 'category_id', 'category_desc',
       'amsl', 'amsl_unit', 'org_title', 'org_content_id', 'addresses_count',
       'locality', 'region', 'postalcode', 'primary_loc', 'display_url',
       'recurrence_type', 'hours', 'created_date', 'last_modified_date',
       'start_date_date', 'end_date_date', 'status', 'Latitude', 'Longitude',
       'Community Board', 'Community Council ', 'Census Tract', 'BIN', 'BBL',
       'NTA']
volunteer = pd.read_csv('data/volunteer_opportunities.csv', 
                            usecols=vol_cols)

In [3]:
volunteer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   opportunity_id      665 non-null    int64  
 1   content_id          665 non-null    int64  
 2   vol_requests        665 non-null    int64  
 3   event_time          665 non-null    int64  
 4   title               665 non-null    object 
 5   hits                665 non-null    int64  
 6   summary             665 non-null    object 
 7   is_priority         62 non-null     object 
 8   category_id         617 non-null    float64
 9   category_desc       617 non-null    object 
 10  amsl                0 non-null      float64
 11  amsl_unit           0 non-null      float64
 12  org_title           665 non-null    object 
 13  org_content_id      665 non-null    int64  
 14  addresses_count     665 non-null    int64  
 15  locality            595 non-null    object 
 16  region  

In [4]:
# Check how many values are missing in the category_desc column
print(volunteer['category_desc'].isnull().sum())

# Subset the volunteer dataset
volunteer_subset = volunteer[volunteer['category_desc'].notnull()]

# Print out the shape of the subset
print(volunteer_subset.shape)

48
(617, 35)


### Converting a column type

In [5]:
# Print the head of the hits column
print(volunteer["hits"].head())

# Convert the hits column to type int
volunteer["hits"] = volunteer["hits"].astype('int64')

# Look at the dtypes of the dataset
print(volunteer.hits.dtype)

0    737
1     22
2     62
3     14
4     31
Name: hits, dtype: int64
int64


### Stratified sampling
We know that the distribution of variables in the category_desc column in the volunteer dataset is uneven. If we wanted to train a model to try to predict category_desc, we would want to train the model on a sample of data that is representative of the entire dataset. Stratified sampling is a way to achieve this.

#### Instructions

- Create a volunteer_X dataset with all of the columns except category_desc.
- Create a volunteer_y training labels dataset.
- Split up the volunteer_X dataset using scikit-learn's train_test_split function and passing volunteer_y into the stratify= parameter.
- Take a look at the category_desc value counts on the training labels.

In [6]:
volunteer.fillna(123456789, inplace=True)
# Create a data with all columns except category_desc
volunteer_X = volunteer.drop('category_desc', axis=1)

# Create a category_desc labels dataset
volunteer_y = volunteer[['category_desc']]

# Use stratified sampling to split up the dataset according to the volunteer_y dataset
X_train, X_test, y_train, y_test = train_test_split(volunteer_X,
                                    volunteer_y, stratify = volunteer_y)
X_train = X_train.replace(123456789, np.nan) 
X_test = X_test.replace(123456789, np.nan)
y_train = y_train.replace(123456789, np.nan)
y_test = y_test.replace(123456789, np.nan)
volunteer = volunteer.replace(123456789, np.nan)

# Print out the category_desc counts on the training y labels
print(y_train['category_desc'].value_counts())

Strengthening Communities    230
Helping Neighbors in Need     89
Education                     69
Health                        39
Environment                   24
Emergency Preparedness        11
Name: category_desc, dtype: int64


# 2. Standardizing Data

This chapter is all about standardizing data. Often a model will make some assumptions about the distribution or scale of your features. Standardization is a way to make your data fit these assumptions and improve the algorithm's performance

### Modeling without normalizing
Let's take a look at what might happen to your model's accuracy if you try to model data without doing some sort of standardization first. Here we have a subset of the wine dataset. One of the columns, Proline, has an extremely high variance compared to the other columns. This is an example of where a technique like log normalization would come in handy, which you'll learn about in the next section.

The scikit-learn model training process should be familiar to you at this point, so we won't go too in-depth with it. You already have a k-nearest neighbors model available (knn) as well as the X and y sets you need to fit and score on.

#### Instructions

- Split up the X and y sets into training and test sets using train_test_split().
- Use the knn model's fit() method on the X_train data and y_train labels, to fit the model to the data.
- Print out the knn model's score() on the X_test data and y_test labels to evaluate the model.

In [7]:
wine = pd.read_csv('data/wine_types.csv')
X = wine.drop('Type', axis = 1)
y = wine.Type
knn  = KNeighborsClassifier()

In [8]:
# Split the dataset and labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Fit the k-nearest neighbors model to the training data
knn.fit(X_train, y_train)

# Score the model on the test data
print(knn.score(X_test,y_test))

0.6222222222222222


### Log normalization in Python

In [9]:
# Print out the variance of the Proline column
print(wine.Proline.var())

# Apply the log normalization function to the Proline column
wine['Proline_log'] = np.log(wine.Proline)

# Check the variance of the normalized Proline column
print(wine.Proline_log.var())

99166.71735542428
0.17231366191842018


### Scaling data - standardizing columns
Since we know that the Ash, Alcalinity of ash, and Magnesium columns in the wine dataset are all on different scales, let's standardize them in a way that allows for use in a linear model.

#### Instructions

- Import StandardScaler from sklearn.preprocessing.
- Create the StandardScaler() method and store in a variable named ss.
- Create a subset of the wine DataFrame of the Ash, Alcalinity of ash, and - Magnesium columns, store in a variable named wine_subset.
- Apply the ss.fit_transform method to the wine_subset DataFrame.

In [10]:
# Create the scaler
ss = StandardScaler()

# Take a subset of the DataFrame you want to scale 
wine_subset = wine[['Ash', 'Alcalinity of ash',  'Magnesium']]

# Apply the scaler to the DataFrame subset
wine_subset_scaled = ss.fit_transform(wine_subset)

### KNN on non-scaled data
Let's first take a look at the accuracy of a K-nearest neighbors model on the wine dataset without standardizing the data. The knn model as well as the X and y data and labels sets have been created already. Most of this process of creating models in scikit-learn should look familiar to you.

#### Instructions

- Split the dataset into training and test sets using train_test_split().
- Use the knn model's fit() method on the X_train data and y_train labels, to fit the model to the data.
- Print out the knn model's score() on the X_test data and y_test labels to evaluate the model.


In [11]:
# Split the dataset and labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Fit the k-nearest neighbors model to the training data
knn.fit(X_train, y_train)

# Score the model on the test data
print(knn.score(X_test, y_test))

0.5777777777777777


### KNN on scaled data
The accuracy score on the unscaled wine dataset was decent, but we can likely do better if we scale the dataset. The process is mostly the same as the previous exercise, with the added step of scaling the data. Once again, the knn model as well as the X and y data and labels set have already been created for you.

#### Instructions

- Create the StandardScaler() method, stored in a variable named ss.
- Apply the ss.fit_transform method to the X dataset.
- Use the knn model's fit() method on the X_train data and y_train labels, to fit the model to the data.
- Print out the knn model's score() on the X_test data and y_test labels to evaluate the model.


In [12]:
# Create the scaling method.
ss = StandardScaler()

# Apply the scaling method to the dataset used for modeling.
X_scaled = ss.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

# Fit the k-nearest neighbors model to the training data.
knn.fit(X_train,y_train)

# Score the model on the test data.
print(knn.score(X_test,y_test))

0.9555555555555556


# 3. Feature Engineering

In this section you'll learn about feature engineering. You'll explore different ways to create new, more useful, features from the ones already in your dataset. You'll see how to encode, aggregate, and extract information from both numerical and textual features.

### Encoding categorical variables - binary
Take a look at the hiking dataset. There are several columns here that need encoding, one of which is the Accessible column, which needs to be encoded in order to be modeled. Accessible is a binary feature, so it has two values - either Y or N - so it needs to be encoded into 1s and 0s. Use scikit-learn's LabelEncoder method to do that transformation.

#### Instructions

- Store LabelEncoder() in a variable named enc
- Using the encoder's fit_transform() function, encode the hiking dataset's "Accessible" column. Call the new column Accessible_enc.
- Compare the two columns side-by-side to see the encoding.

In [13]:
hiking = pd.read_json('data/hiking.json')

In [14]:
# Set up the LabelEncoder object
enc = LabelEncoder()

# Apply the encoding to the "Accessible" column
hiking['Accessible_enc'] = enc.fit_transform(hiking.Accessible)

# Compare the two columns
print(hiking[['Accessible_enc', 'Accessible']].head())

   Accessible_enc Accessible
0               1          Y
1               0          N
2               0          N
3               0          N
4               0          N


### Encoding categorical variables - one-hot
One of the columns in the volunteer dataset, category_desc, gives category descriptions for the volunteer opportunities listed. Because it is a categorical variable with more than two categories, we need to use one-hot encoding to transform this column numerically. Use Pandas' get_dummies() function to do so.

#### Instructions

- Call get_dummies() on the volunteer["category_desc"] column to create the encoded columns and assign it to category_enc.
- Print out the head() of the category_enc variable to take a look at the encoded columns.

In [15]:
# Transform the category_desc column
category_enc = pd.get_dummies(volunteer.category_desc)

# Take a look at the encoded columns
print(category_enc.head())

   Education  Emergency Preparedness  Environment  Health  \
0          0                       0            0       0   
1          0                       0            0       0   
2          0                       0            0       0   
3          0                       0            0       0   
4          0                       0            1       0   

   Helping Neighbors in Need  Strengthening Communities  
0                          0                          0  
1                          0                          1  
2                          0                          1  
3                          0                          1  
4                          0                          0  


### Engineering numerical features - taking an average
A good use case for taking an aggregate statistic to create a new feature is to take the mean of columns. Here, you have a DataFrame of running times named running_times_5k. For each name in the dataset, take the mean of their 5 run times.

#### Instructions

- Create a list of the columns you want to take the average of and store it in a variable named run_columns.
- Use apply to take the mean() of the list of columns and remember to set axis=1. Use lambda row: in the apply.
- Print out the DataFrame to see the mean column.

In [16]:
running_dic = {'name': ['Sue', 'Mark', 'Sean', 'Erin', 'Jenny', 'Russell'],
             'run1': [20.1, 16.5, 23.5, 21.7, 25.8, 30.9],
             'run2': [18.5, 17.1, 25.1, 21.1, 27.1, 29.6],
             'run3': [19.6, 16.9, 25.2, 20.9, 26.1, 31.4],
             'run4': [20.3, 17.6, 24.6, 22.1, 26.7, 30.4],
             'run5': [18.3, 17.3, 23.9, 22.2, 26.9, 29.9]}
running_times_5k = pd.DataFrame(running_dic)

In [17]:
# Create a list of the columns to average
run_columns = [col for col in running_times_5k.columns  if running_times_5k[col].dtype == 'float64']

# Use apply to create a mean column
running_times_5k["mean"] = running_times_5k.apply(lambda row: row[run_columns].mean(), axis=1)

# Take a look at the results
print(running_times_5k)

      name  run1  run2  run3  run4  run5   mean
0      Sue  20.1  18.5  19.6  20.3  18.3  19.36
1     Mark  16.5  17.1  16.9  17.6  17.3  17.08
2     Sean  23.5  25.1  25.2  24.6  23.9  24.46
3     Erin  21.7  21.1  20.9  22.1  22.2  21.60
4    Jenny  25.8  27.1  26.1  26.7  26.9  26.52
5  Russell  30.9  29.6  31.4  30.4  29.9  30.44


### Engineering numerical features - datetime
There are several columns in the volunteer dataset comprised of datetimes. Let's take a look at the start_date_date column and extract just the month to use as a feature for modeling.

#### Instructions

- Use Pandas to_datetime() function on the volunteer["start_date_date"] column and store it in a new column called start_date_converted.
- To retrieve just the month, apply a lambda function to volunteer["start_date_converted"] that grabs the .month attribute from the row. Store this in a new column called start_date_month.
- Print the head() of just the start_date_converted and start_date_month columns.

In [18]:
# First, convert string column to date column
volunteer["start_date_converted"] = pd.to_datetime(volunteer["start_date_date"])

# Extract just the month from the converted column
volunteer["start_date_month"] = volunteer["start_date_converted"].apply(lambda row: row.month)

# Take a look at the converted and new month columns
print(volunteer[["start_date_converted", "start_date_month"]].head())


  start_date_converted  start_date_month
0           2011-07-30                 7
1           2011-02-01                 2
2           2011-01-29                 1
3           2011-02-14                 2
4           2011-02-05                 2


### Engineering features from strings - extraction
The Length column in the hiking dataset is a column of strings, but contained in the column is the mileage for the hike. We're going to extract this mileage using regular expressions, and then use a lambda in Pandas to apply the extraction to the DataFrame.

#### Instructions

- Create a pattern that will extract numbers and decimals from text, using \d+ to get numbers and \. to get decimals, and pass it into re's compile function.
- Use re's match function to search the text, passing in the pattern and the length text.
- Use the matched mile's group() attribute to extract the matched pattern, making sure to match group 0, and pass it into float.
- Apply the return_mileage() function to the hiking["Length"] column.

In [19]:
import re
# Write a pattern to extract numbers and decimals
def return_mileage(length):
    if type(length)!= str:
        length = str(length)
    pattern = re.compile(r"\d+\.\d+")
    
    # Search the text for matches
    mile = re.match(pattern, length)
    
    # If a value is returned, use group(0) to return the found value
    if mile is not None:
        return float(mile.group(0))
        
# Apply the function to the Length column and take a look at both columns
hiking["Length_num"] = hiking["Length"].apply(lambda row: return_mileage(row))
print(hiking[["Length", "Length_num"]].head())

       Length  Length_num
0   0.8 miles        0.80
1    1.0 mile        1.00
2  0.75 miles        0.75
3   0.5 miles        0.50
4   0.5 miles        0.50


### Engineering features from strings - tf/idf
Let's transform the volunteer dataset's title column into a text vector, to use in a prediction task in the next exercise.

#### Instructions

- Store the volunteer["title"] column in a variable named title_text.
- Use the tfidf_vec vectorizer's fit_transform() function on title_text to transform the text into a tf-idf vector.

In [20]:
volunteer = volunteer[~volunteer.category_desc.isna()]
# Take the title text
title_text = volunteer["title"]

# Create the vectorizer method
tfidf_vec = TfidfVectorizer()

# Transform the text into tf-idf vectors
text_tfidf = tfidf_vec.fit_transform(title_text)

### Text classification using tf/idf vectors
Now that we've encoded the volunteer dataset's title column into tf/idf vectors, let's use those vectors to try to predict the category_desc column.

#### Instructions

- Using train_test_split, split the text_tfidf vector, along with your y variable, into training and test sets. Set the stratify parameter equal to y, since the class distribution is uneven. Notice that we have to run the toarray() method on the tf/idf vector, in order to get in it the proper format for scikit-learn.
- Use Naive Bayes' fit() method on the X_train and y_train variables.
- Print out the score() of the X_test and y_test variables.


In [21]:
# Split the dataset according to the class distribution of category_desc
y = volunteer["category_desc"]
X_train, X_test, y_train, y_test = train_test_split(text_tfidf.toarray(), y, stratify=y)

nb = GaussianNB()
# Fit the model to the training data
nb.fit(X_train, y_train)

# Print out the model's accuracy
print(nb.score(X_test, y_test))

0.5096774193548387


# 4. Selecting features for modeling

This chapter goes over a few different techniques for selecting the most important features from your dataset. You'll learn how to drop redundant features, work with text vectors, and reduce the number of features in your dataset using principal component analysis (PCA).

### Modeling without normalizing
Let's take a look at what might happen to your model's accuracy if you try to model data without doing some sort of standardization first. Here we have a subset of the wine dataset. One of the columns, Proline, has an extremely high variance compared to the other columns. This is an example of where a technique like log normalization would come in handy, which you'll learn about in the next section.

The scikit-learn model training process should be familiar to you at this point, so we won't go too in-depth with it. You already have a k-nearest neighbors model available (knn) as well as the X and y sets you need to fit and score on.

#### Instructions

- Split up the X and y sets into training and test sets using train_test_split().
- Use the knn model's fit() method on the X_train data and y_train labels, to fit the model to the data.
- Print out the knn model's score() on the X_test data and y_test labels to evaluate the model.

In [22]:
X = wine.drop('Type', axis = 1)
y= wine.Type
# Split the dataset and labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

knn = KNeighborsClassifier()
# Fit the k-nearest neighbors model to the training data
knn.fit(X_train, y_train)

# Score the model on the test data
print(knn.score(X_test,y_test))

0.6888888888888889


### Selecting relevant features
Now let's identify the redundant columns in the volunteer dataset and perform feature selection on the dataset to return a DataFrame of the relevant features.

For example, if you explore the volunteer dataset in the console, you'll see three features which are related to location: locality, region, and postalcode. They contain repeated information, so it would make sense to keep only one of the features.

There are also features that have gone through the feature engineering process: columns like Education and Emergency Preparedness are a product of encoding the categorical variable category_desc, so category_desc itself is redundant now.

Take a moment to examine the features of volunteer in the console, and try to identify the redundant features.

#### Instructions

- Create a list of redundant column names and store it in the to_drop variable:
- Out of all the location-related features, keep only postcode.
- Features that have gone through the feature engineering process are redundant as well.
- Drop the columns from the dataset using .drop().
- Print out the .head() of the DataFrame to see the selected columns.


In [23]:
volunteer = pd.read_csv('data/volunteer.csv')

In [24]:
# Create a list of redundant column names to drop
to_drop = ["category_desc", "created_date", "locality", "region", "vol_requests"]

# Drop those columns from the dataset
volunteer_subset = volunteer.drop(to_drop, axis=1)

# Print out the head of the new dataset
print(volunteer_subset.head())

                                               title  hits  postalcode  \
0                                       Web designer    22     10010.0   
1      Urban Adventures - Ice Skating at Lasker Rink    62     10026.0   
2  Fight global hunger and support women farmers ...    14      2114.0   
3                                      Stop 'N' Swap    31     10455.0   
4                               Queens Stop 'N' Swap   135     11372.0   

   vol_requests_lognorm  created_month  Education  Emergency Preparedness  \
0              0.693147              1          0                       0   
1              2.995732              1          0                       0   
2              6.214608              1          0                       0   
3              2.708050              1          0                       0   
4              2.708050              1          0                       0   

   Environment  Health  Helping Neighbors in Need  Strengthening Communities  
0            

### Checking for correlated features
Let's take a look at the wine dataset again, which is made up of continuous, numerical features. Run Pearson's correlation coefficient on the dataset to determine which columns are good candidates for eliminating. Then, remove those columns from the DataFrame.

#### Instructions

- Print out the column correlations of the wine dataset using corr().
- Take a minute to look at the correlations. Identify a column where the correlation value is greater than 0.75 at least twice and store it in the to_drop variable.
- Drop that column from the DataFrame using drop().

In [25]:
wine1 = wine[['Flavanoids','Total phenols', 'Malic acid', 'OD280/OD315 of diluted wines', 'Hue' ]]

In [26]:
# Print out the column correlations of the wine dataset
print(wine1.corr())

# Take a minute to find the column where the correlation value is greater than 0.75 at least twice
to_drop = "Flavanoids"

# Drop that column from the DataFrame
wine1= wine1.drop(to_drop, axis = 1)

                              Flavanoids  Total phenols  Malic acid  \
Flavanoids                      1.000000       0.864564   -0.411007   
Total phenols                   0.864564       1.000000   -0.335167   
Malic acid                     -0.411007      -0.335167    1.000000   
OD280/OD315 of diluted wines    0.787194       0.699949   -0.368710   
Hue                             0.543479       0.433681   -0.561296   

                              OD280/OD315 of diluted wines       Hue  
Flavanoids                                        0.787194  0.543479  
Total phenols                                     0.699949  0.433681  
Malic acid                                       -0.368710 -0.561296  
OD280/OD315 of diluted wines                      1.000000  0.565468  
Hue                                               0.565468  1.000000  


### Exploring text vectors, part 1
Let's expand on the text vector exploration method we just learned about, using the volunteer dataset's title tf/idf vectors. In this first part of text vector exploration, we're going to add to that function we learned about in the slides. We'll return a list of numbers with the function. In the next exercise, we'll write another function to collect the top words across all documents, extract them, and then use that list to filter down our text_tfidf vector.

#### Instructions

- Add parameters called original_vocab, for the tfidf_vec.vocabulary_, and top_n.
- Call pd.Series on the zipped dictionary. This will make it easier to operate on.
- Use the sort_values function to sort the series and slice the index up to top_n words.
- Call the function, setting original_vocab=tfidf_vec.vocabulary_, setting vector_index=8 to grab the 9th row, and setting top_n=3, to grab the top 3 weighted words.


In [27]:
vocab = {v:k for k,v in tfidf_vec.vocabulary_.items()}

In [28]:
# Add in the rest of the parameters
def return_weights(vocab, original_vocab, vector, vector_index, top_n):
    zipped = dict(zip(vector[vector_index].indices, vector[vector_index].data))
    
    # Let's transform that zipped dict into a series
    zipped_series = pd.Series({vocab[i]:zipped[i] for i in vector[vector_index].indices})
    
    # Let's sort the series to pull out the top n weighted words
    zipped_index = zipped_series.sort_values(ascending=False)[:top_n].index
    return [original_vocab[i] for i in zipped_index]

# Print out the weighted words
print(return_weights(vocab, tfidf_vec.vocabulary_, text_tfidf, 8, 3))

[189, 942, 466]


### Exploring text vectors, part 2
Using the function we wrote in the previous exercise, we're going to extract the top words from each document in the text vector, return a list of the word indices, and use that list to filter the text vector down to those top words.

### Instructions

- Call return_weights to return the top weighted words for that document.
- Call set on the returned filter_list so we don't get duplicated numbers.
- Call words_to_filter, passing in the following parameters: vocab for the vocab parameter, tfidf_vec.vocabulary_ for the original_vocab parameter, text_tfidf for the vector parameter, and 3 to grab the top_n 3 weighted words from each document.
- Finally, pass that filtered_words set into a list to use as a filter for the text vector.


In [29]:
def words_to_filter(vocab, original_vocab, vector, top_n):
    filter_list = []
    for i in range(0, vector.shape[0]):
    
        # Here we'll call the function from the previous exercise, and extend the list we're creating
        filtered = return_weights(vocab, original_vocab, vector, i, top_n)
        filter_list.extend(filtered)
    # Return the list in a set, so we don't get duplicate word indices
    return set(filter_list)

# Call the function to get the list of word indices
filtered_words = words_to_filter(vocab, tfidf_vec.vocabulary_, text_tfidf, 3)

# By converting filtered_words back to a list, we can use it to filter the columns in the text vector
filtered_text = text_tfidf[:, list(filtered_words)]

### Training Naive Bayes with feature selection
Let's re-run the Naive Bayes text classification model we ran at the end of chapter 3, with our selection choices from the previous exercise, on the volunteer dataset's title and category_desc columns.

### Instructions
- Use train_test_split on the filtered_text text vector, the y labels (which is the category_desc labels), and pass the y set to the stratify parameter, since we have an uneven class distribution.
- Fit the nb Naive Bayes model to train_X and train_y.
- Score the nb model on the test_X and test_y test sets.

In [30]:
y = volunteer.category_desc
# Split the dataset according to the class distribution of category_desc, using the filtered_text vector
train_X, test_X, train_y, test_y = train_test_split(filtered_text.toarray(), y, stratify=y)

# Fit the model to the training data
nb.fit(train_X, train_y)

# Print out the model's accuracy
print(nb.score(test_X, test_y))

0.5419354838709678


### Using PCA
Let's apply PCA to the wine dataset, to see if we can get an increase in our model's accuracy.

#### Instructions

- Set up the PCA object. You'll use PCA on the wine dataset minus its label for Type, stored in the variable wine_X.
- Apply PCA to wine_X using pca's fit_transform method and store the transformed vector in transformed_X.
- Print out the explained_variance_ratio_ attribute of pca to check how much variance is explained by each component.

In [31]:
wine.drop('Proline_log', axis=1)

# Set up PCA and the X vector for diminsionality reduction
pca = PCA()
wine_X = wine.drop("Type", axis=1)
y = wine.Type

# Apply PCA to the wine dataset X vector
transformed_X = pca.fit_transform(wine_X)

# Look at the percentage of variance explained by the different components
print(pca.explained_variance_ratio_)

[9.98091157e-01 1.73591574e-03 9.49596845e-05 5.02203105e-05
 1.23683570e-05 8.46366883e-06 2.80684492e-06 1.52331150e-06
 1.13031557e-06 7.22017230e-07 3.79083815e-07 2.12869847e-07
 8.25543070e-08 5.87363612e-08]


### Training a model with PCA
Now that we have run PCA on the wine dataset, let's try training a model with it.

#### Instructions

- Split the transformed_X vector and the y labels set into training and test sets using train_test_split.
- Fit the knn model using the fit() function on the X_wine_train and y_wine_train sets.
- Print out the score using knn's score() function on X_wine_test and y_wine_test.

In [32]:
knn = KNeighborsClassifier()
# Split the transformed X and the y labels into training and test sets
X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(transformed_X, y )

# Fit knn to the training data
knn.fit(X_wine_train, y_wine_train)

# Score knn on the test data and print it out
knn.score(X_wine_test, y_wine_test)

0.8

# 5. Putting it all together

Now that you've learned all about preprocessing you'll try these techniques out on a dataset that records information on UFO sightings.

### Checking column types
Take a look at the UFO dataset's column types using the dtypes attribute. Two columns jump out for transformation: the seconds column, which is a numeric column but is being read in as object, and the date column, which can be transformed into the datetime type. That will make our feature engineering efforts easier later on.

#### Instructions

- Print out the dtypes of the ufo dataset.
- Change the type of the seconds column by passing the float type into the astype() method.
- Change the type of the date column by passing ufo["date"] into the pd.to_datetime() function.
- Print out the dtypes of the seconds and date columns, to make sure it worked.

In [33]:
ufo = pd.read_csv("data/ufo_sightings_large.csv")

In [34]:
# Check the column types
ufo.dtypes

# Change the type of seconds to float
ufo["seconds"] = ufo["seconds"].astype('float')

# Change the date column to type datetime
ufo["date"] = pd.to_datetime(ufo["date"])

# Check the column types
print(ufo[["date","seconds"]].dtypes)

date       datetime64[ns]
seconds           float64
dtype: object


### Dropping missing data
Let's remove some of the rows where certain columns have missing values. We're going to look at the length_of_time column, the state column, and the type column. If any of the values in these columns are missing, we're going to drop the rows.

#### Instructions

- Check how many values are missing in the length_of_time, state, and type columns, using isnull() to check for nulls and sum() to calculate how many exist.
- Use boolean indexing to filter out the rows with those missing values, using notnull() to check the column. Here, we can chain together each column we want to check.
- Print out the shape of the new ufo_no_missing dataset

In [35]:
# Check how many values are missing in the length_of_time, state, and type columns
print(ufo[['length_of_time', 'state', 'type']].isnull().sum())

# Keep only rows where length_of_time, state, and type are not null
ufo_no_missing = ufo[ufo.length_of_time.notnull() & 
          ufo.state.notnull() & 
          ufo['type']]

# Print out the shape of the new dataset
print(ufo_no_missing.shape)

length_of_time    143
state             419
type              159
dtype: int64
(4283, 11)


### Extracting numbers from strings
The length_of_time field in the UFO dataset is a text field that has the number of minutes within the string. Here, you'll extract that number from that text field using regular expressions.

#### Instructions

- Pass \d+ into re.compile() in the pattern variable to designate that we want to grab as many digits as possible from the string.
- Into re.match(), pass the pattern we just created, as well as the time_string we want to extract from.
- Use lambda within the apply() method to perform the extraction.
- Print out the head() of both the length_of_time and minutes columns to compare.

In [36]:
def find(string):
    if type(string) == str:
        return "minutes" in string
    else:
        return False
ufo = ufo[[find(string) for string in ufo.length_of_time]]

In [37]:
def return_minutes(time_string):

    # Use \d+ to grab digits
    pattern = re.compile(r"\d+")
    
    # Use match on the pattern and column
    num = re.match(pattern, time_string)
    if num is not None:
        return int(num.group(0))
        
# Apply the extraction to the length_of_time column
ufo["minutes"] = ufo["length_of_time"].apply(lambda x: 
                                             return_minutes(x))

# Take a look at the head of both of the columns
print(ufo[["length_of_time","minutes"]].head())

     length_of_time  minutes
3   about 5 minutes      NaN
5        10 minutes     10.0
8         2 minutes      2.0
9         2 minutes      2.0
10        5 minutes      5.0


### Identifying features for standardization
In this section, you'll investigate the variance of columns in the UFO dataset to determine which features should be standardized. After taking a look at the variances of the seconds and minutes column, you'll see that the variance of the seconds column is extremely high. Because seconds and minutes are related to each other (an issue we'll deal with when we select features for modeling), let's log normlize the seconds column.

#### Instructions

- Use the var() method on the seconds and minutes columns to check the variance. Notice how high the variance is on the seconds column.
- Using np.log() perform log normalization on the seconds column, transforming it into a new column named seconds_log.
- Print out the variance of the seconds_log column.

In [38]:
# Check the variance of the seconds and minutes columns
print(ufo[["seconds", "minutes"]].var())

# Log normalize the seconds column
ufo["seconds_log"] = np.log(ufo.seconds)

# Print out the variance of just the seconds_log column
print(ufo["seconds_log"].var())

seconds    446629.150501
minutes       122.898997
dtype: float64
0.8547283011401443


### Encoding categorical variables
There are couple of columns in the UFO dataset that need to be encoded before they can be modeled through scikit-learn. You'll do that transformation here, using both binary and one-hot encoding methods.

#### Instructions

- Using apply(), write a lambda that returns a 1 if the value is us, else return 0. This is something we learned in Chapter 3 if you need a refresher.
- Next, print out the number of unique() values of the type column.
- Using pd.get_dummies(), create a one-hot encoded set of the type column.
- Finally, use pd.concat() to concatenate the ufo dataset to the type_set encoded variables.

In [39]:
# Use Pandas to encode us values as 1 and others as 0
ufo["country_enc"] = ufo["country"].apply(lambda x : int(x == 'us'))

# Print the number of unique type values
print(len(ufo['type'].unique()))

# Create a one-hot encoded set of the type values
type_set = pd.get_dummies(ufo['type'])

# Concatenate this set back to the ufo DataFrame
ufo = pd.concat([ufo, type_set], axis=1)

22


### Features from dates
Another feature engineering task to perform is month and year extraction. Perform this task on the date column of the ufo dataset.

#### Instructions

- Print out the head() of the date column.
- Using apply(), lambda, and the .month attribute, extract the month from the date column.
- Using apply(), lambda, and the .year attribute, extract the year from the date column.
- Take a look at the head() of the date, month, and year columns.


In [40]:
# Look at the first 5 rows of the date column
print(ufo.date.head())

# Extract the month from the date column
ufo["month"] = ufo["date"].apply(lambda date : date.month)

# Extract the year from the date column
ufo["year"] = ufo["date"].apply(lambda date : date.year)

# Take a look at the head of all three columns
print(ufo[['date','month', 'year']])

3    2002-11-21 05:45:00
5    2012-06-16 23:00:00
8    2013-06-09 00:00:00
9    2013-04-26 23:27:00
10   2013-09-13 20:30:00
Name: date, dtype: datetime64[ns]
                    date  month  year
3    2002-11-21 05:45:00     11  2002
5    2012-06-16 23:00:00      6  2012
8    2013-06-09 00:00:00      6  2013
9    2013-04-26 23:27:00      4  2013
10   2013-09-13 20:30:00      9  2013
...                  ...    ...   ...
4926 2002-08-20 23:10:00      8  2002
4927 2013-07-04 21:00:00      7  2013
4928 2008-11-10 00:30:00     11  2008
4929 1998-06-25 22:18:00      6  1998
4934 2005-12-10 18:00:00     12  2005

[1755 rows x 3 columns]


### Text vectorization
Let's transform the desc column in the UFO dataset into tf/idf vectors, since there's likely something we can learn from this field.

#### Instructions

- Print out the head() of the ufo["desc"] column.
- Set vec equal to the TfidfVectorizer() object.
- Use vec's fit_transform() method on the ufo["desc"] column.
- Print out the shape of the desc_tfidf vector, to take a look at the number of columns this created. The output is in the shape (rows, columns).

In [41]:
# Take a look at the head of the desc field
print(ufo["desc"].head())

# Create the tfidf vectorizer object
vec = TfidfVectorizer()

# Use vec's fit_transform method on the desc field
desc_tfidf = vec.fit_transform(ufo["desc"])

# Look at the number of columns this creates
print(desc_tfidf.shape)

3     It was a large&#44 triangular shaped flying ob...
5     Dancing lights that would fly around and then ...
8     Brilliant orange light or chinese lantern at o...
9     Bright red light moving north to north west fr...
10    North-east moving south-west. First 7 or so li...
Name: desc, dtype: object
(1755, 3330)


### Selecting the ideal dataset
Let's get rid of some of the unnecessary features. Because we have an encoded country column, country_enc, keep it and drop other columns related to location: city, country, lat, long, state.

We have columns related to month and year, so we don't need the date or recorded columns.

We vectorized desc, so we don't need it anymore. For now we'll keep type.

We'll keep seconds_log and drop seconds and minutes.

Let's also get rid of the length_of_time column, which is unnecessary after extracting minutes.

#### Instructions

- Use .corr() to run the correlation on seconds, seconds_log, and minutes in the ufo DataFrame.
- Make a list of columns to drop, in alphabetical order.
- Use drop() to drop the columns.
- Use the words_to_filter() function we created previously. Pass in vocab, vec.vocabulary_, desc_tfidf, and let's keep the top 4 words as the last parameter.

In [42]:
vocab = {v:k for k,v in vec.vocabulary_.items()}

In [43]:
# Check the correlation between the seconds, seconds_log, and minutes columns
print(ufo[['seconds','seconds_log','minutes']].corr())

# Make a list of features to drop
to_drop = ['city', 'country', 'lat', 'long', 'state','date','recorded', 'desc', 'minutes', 'seconds', 'length_of_time']

# Drop those features
ufo_dropped = ufo.drop(to_drop, axis = 1)

# Let's also filter some words out of the text vector we created
filtered_words = words_to_filter(vocab, vec.vocabulary_, desc_tfidf, 4)

              seconds  seconds_log   minutes
seconds      1.000000     0.884759  0.988714
seconds_log  0.884759     1.000000  0.867706
minutes      0.988714     0.867706  1.000000


### Modeling the UFO dataset, part 1
In this exercise, we're going to build a k-nearest neighbor model to predict which country the UFO sighting took place in. Our X dataset has the log-normalized seconds column, the one-hot encoded type columns, as well as the month and year when the sighting took place. The y labels are the encoded country column, where 1 is us and 0 is ca.

#### Instructions

- Print out the .columns of the X set.
- Split up the X and y sets using train_test_split(). Pass the y set to the stratify= parameter, since we have imbalanced classes here.
- Use fit() to fit train_X and train_y.
- Print out the .score() of the knn model on the test_X and test_y sets.

In [50]:
X = ufo_dropped.drop(['country_enc', 'type'], axis = 1)
y = ufo_dropped.country_enc

In [52]:
# Take a look at the features in the X set of data
print(X.columns)

# Split the X and y sets using train_test_split, setting stratify=y
train_X, test_X, train_y, test_y = train_test_split(X, y ,stratify = y)

# Fit knn to the training sets
knn.fit(train_X, train_y)

# Print the score of knn on the test sets
print(knn.score(test_X, test_y))

Index(['seconds_log', 'changing', 'chevron', 'cigar', 'circle', 'cone',
       'cross', 'cylinder', 'diamond', 'disk', 'egg', 'fireball', 'flash',
       'formation', 'light', 'other', 'oval', 'rectangle', 'sphere',
       'teardrop', 'triangle', 'unknown', 'month', 'year'],
      dtype='object')
0.8109339407744874


### Modeling the UFO dataset, part 2
Finally, let's build a model using the text vector we created, desc_tfidf, using the filtered_words list to create a filtered text vector. Let's see if we can predict the type of the sighting based on the text. We'll use a Naive Bayes model for this.

#### Instructions

- On the desc_tfidf vector, filter by passing a list of filtered_words into the index.
- Split up the X and y sets using train_test_split(). Remember to convert filtered_text using toarray(). Pass the y set to the stratify= parameter, since we have imbalanced classes here.
- Use the nb model's fit() to fit train_X and train_y.
- Print out the .score() of the nb model on the test_X and test_y sets.

In [53]:
# Use the list of filtered words we created to filter the text vector
filtered_text = desc_tfidf[:, list(filtered_words)]

# Split the X and y sets using train_test_split, setting stratify=y 
train_X, test_X, train_y, test_y = train_test_split(filtered_text.toarray(), y, stratify=y)

# Fit nb to the training sets
nb.fit(train_X,train_y)

# Print the score of nb on the test sets
print(nb.score(test_X, test_y))


0.662870159453303
